In [4]:
pip install prophet

     --------------------------------------- 12.9/12.9 MB 38.4 MB/s eta 0:00:00
  Using cached convertdate-2.4.0-py3-none-any.whl (47 kB)
  Using cached holidays-0.33-py3-none-any.whl (759 kB)
  Using cached LunarCalendar-0.0.9-py2.py3-none-any.whl (18 kB)
     ---------------------------------------- 83.2/83.2 kB 4.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Step 1.2:Loading libraries pandas and numpy and giving them short names pd and np. These aliases are industry-standard.
# Same for numpy, scikit-learn and statsmodels.
import pandas as pd
import numpy as np
import os.path as osp
import os
os.chdir('C:/Users/anton/MMA 867')

#Sarah_raw_data = pd.read_excel(data_path,sheet_name='Raw Data')
data_path = osp.join(osp.curdir,'Data','Filltered_notnewcluster (6).csv')
df = pd.read_csv(data_path)

In [2]:
df.head()

,code,Date,Month,Year,Monthly Sold,Category,Shelf_Life_9,Shelf_Life_10,Shelf_Life_12,Shelf_Life_18,Shelf_Life_24,Type_Normal,Type_Signature,Cluster
0,C00003,2020-09-30,9,2020,234.0,Skin_Care,0,0,0,0,0,0,1,2
1,C00003,2020-10-31,10,2020,215.0,Skin_Care,0,0,0,0,0,0,1,2
2,C00003,2020-11-30,11,2020,414.0,Skin_Care,0,0,0,0,0,0,1,2
3,C00003,2020-12-31,12,2020,187.0,Skin_Care,0,0,0,0,0,0,1,2
4,C00003,2021-01-31,1,2021,92.0,Skin_Care,0,0,0,0,0,0,1,2


In [10]:
# Import necessary libraries
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Function to transform the date to the first day of the month
def transform_date_to_first_day_of_month(date):
    return date.replace(day=1)

# Apply date transformation
df['Date'] = pd.to_datetime(df['Date'])
df['Date'] = df['Date'].apply(transform_date_to_first_day_of_month)

# Define rolling window parameters
training_window_size = 24  # 24 months training
testing_window_size = 1    # 1 month testing

# Lists to store metrics for each rolling window
results = []

# Get unique codes
unique_codes = df['code'].unique()

# Perform rolling window cross-validation for each unique code
for code in unique_codes:
    code_data = df[df['code'] == code]
    
    for start_idx in range(0, len(code_data) - training_window_size - testing_window_size + 1):
        train_start_idx = start_idx
        train_end_idx = start_idx + training_window_size
        test_start_idx = train_end_idx
        test_end_idx = test_start_idx + testing_window_size
    
        # Split data into training and testing sets
        train_data = code_data.iloc[train_start_idx:train_end_idx]
        test_data = code_data.iloc[test_start_idx:test_end_idx]
    
        # Create a Prophet model
        model = Prophet()
        train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
    
        # Fit the model on training data
        model.fit(train_data)
    
        # Create a future dataframe for forecasting
        future = model.make_future_dataframe(periods=testing_window_size, freq="M")
    
        # Forecast for the next 1 month
        forecast = model.predict(future)
    
        # Extract actual and forecasted values for the testing period
        actual_values = test_data["Monthly Sold"].values
        forecasted_values = forecast["yhat"].tail(testing_window_size).values
    
        # Calculate performance metrics (MAE, MAPE, RMSE)
        mae = mean_absolute_error(actual_values, forecasted_values)
        mape = np.mean(np.abs((actual_values - forecasted_values) / actual_values)) * 100
        rmse = np.sqrt(mean_squared_error(actual_values, forecasted_values))
    
        # Append metrics to the results list
        results.append({
            "code": code,
            "Date": test_data.iloc[0]["Date"],
            "Actual": actual_values[0],
            "Forecasted": forecasted_values[0],
            "MAE": mae,
            "MAPE": mape,
            "RMSE": rmse
        })

# Export the results to a CSV file
result_df = pd.DataFrame(results)
result_df.to_csv('rolling_window_results.csv', index=False)

C:\Users\anton\AppData\Local\Temp\ipykernel_15500\510362733.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
12:11:47 - cmdstanpy - INFO - Chain [1] start processing
12:11:48 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\anton\AppData\Local\Temp\ipykernel_15500\510362733.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
12:11:48 - cmdstanpy - INFO - Chain [1] start processing
12:11:48 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\anton\AppDa

12:11:51 - cmdstanpy - INFO - Chain [1] start processing
12:11:51 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\anton\AppData\Local\Temp\ipykernel_15500\510362733.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
12:11:51 - cmdstanpy - INFO - Chain [1] start processing
12:11:51 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\anton\AppData\Local\Temp\ipykernel_15500\510362733.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
12:11:51 - cmdstanpy

12:11:54 - cmdstanpy - INFO - Chain [1] start processing
12:11:54 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\anton\AppData\Local\Temp\ipykernel_15500\510362733.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
12:11:54 - cmdstanpy - INFO - Chain [1] start processing
12:11:54 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\anton\AppData\Local\Temp\ipykernel_15500\510362733.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
12:11:54 - cmdstanpy

12:11:57 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\anton\AppData\Local\Temp\ipykernel_15500\510362733.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
12:11:57 - cmdstanpy - INFO - Chain [1] start processing
12:11:57 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\anton\AppData\Local\Temp\ipykernel_15500\510362733.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
12:11:57 - cmdstanpy - INFO - Chain [1] start processing
12:11:57 - cmdstanpy

12:12:00 - cmdstanpy - INFO - Chain [1] start processing
12:12:00 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\anton\AppData\Local\Temp\ipykernel_15500\510362733.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
12:12:00 - cmdstanpy - INFO - Chain [1] start processing
12:12:00 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\anton\AppData\Local\Temp\ipykernel_15500\510362733.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
12:12:00 - cmdstanpy

...

12:13:48 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\anton\AppData\Local\Temp\ipykernel_15500\510362733.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
12:13:48 - cmdstanpy - INFO - Chain [1] start processing
12:13:48 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\anton\AppData\Local\Temp\ipykernel_15500\510362733.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
12:13:48 - cmdstanpy - INFO - Chain [1] start processing
12:13:48 - cmdstanpy

12:13:51 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\anton\AppData\Local\Temp\ipykernel_15500\510362733.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
12:13:51 - cmdstanpy - INFO - Chain [1] start processing
12:13:51 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\anton\AppData\Local\Temp\ipykernel_15500\510362733.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
12:13:51 - cmdstanpy - INFO - Chain [1] start processing
12:13:51 - cmdstanpy

12:13:54 - cmdstanpy - INFO - Chain [1] start processing
12:13:54 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\anton\AppData\Local\Temp\ipykernel_15500\510362733.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
12:13:54 - cmdstanpy - INFO - Chain [1] start processing
12:13:54 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\anton\AppData\Local\Temp\ipykernel_15500\510362733.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
12:13:54 - cmdstanpy

12:13:57 - cmdstanpy - INFO - Chain [1] start processing
12:13:57 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\anton\AppData\Local\Temp\ipykernel_15500\510362733.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
12:13:57 - cmdstanpy - INFO - Chain [1] start processing
12:13:57 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\anton\AppData\Local\Temp\ipykernel_15500\510362733.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
12:13:57 - cmdstanpy

12:14:00 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\anton\AppData\Local\Temp\ipykernel_15500\510362733.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
12:14:00 - cmdstanpy - INFO - Chain [1] start processing
12:14:00 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\anton\AppData\Local\Temp\ipykernel_15500\510362733.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={"Date": "ds", "Monthly Sold": "y"}, inplace=True)
12:14:00 - cmdstanpy - INFO - Chain [1] start processing
12:14:01 - cmdstanpy